# CityBikes

Send a request to CityBikes for the city of your choice. 

In [12]:

import requests
import json
import pandas as pd
import unidecode
import numpy as np
import pickle

In [13]:
# set city 
city_name = "boston"
print(city_name)

boston


In [14]:
# get response for all networks and normalize into dataframe
response = requests.request('GET',url="http://api.citybik.es/v2/networks")
data_networks = response.json()
df_networks = pd.json_normalize(data_networks['networks'])
print(df_networks)

                                       company  \
0                             [ЗАО «СитиБайк»]   
1               [Urban Infrastructure Partner]   
2                          [Comunicare S.r.l.]   
3                         [Cyclopolis Systems]   
4    [Groundwork, Slough Borough Council, ITS]   
..                                         ...   
607                             [PubliBike AG]   
608                             [PubliBike AG]   
609                             [PubliBike AG]   
610                             [PubliBike AG]   
611      [Citybike Normandie, INURBA Mobility]   

                                       href                         id  \
0              /v2/networks/velobike-moscow            velobike-moscow   
1              /v2/networks/baerum-bysykkel            baerum-bysykkel   
2             /v2/networks/bicincitta-siena           bicincitta-siena   
3          /v2/networks/cyclopolis-maroussi        cyclopolis-maroussi   
4            /v2/networks/cyc

In [15]:
# get list of cities, remove accents, and convert to lowercase
city_list = df_networks['location.city'].values
city_list = np.array([unidecode.unidecode(city).lower() for city in city_list])
print(city_list)

['moscow' 'baerum' 'siena' 'maroussi' 'slough' 'nafplio'
 'parco dei colli di bergamo' 'aigialeia' 'marathon' 'mantova'
 'nea smyrni' 'moschato-tavros' 'arxaia olympia' 'kiato' 'assemini'
 'rhodes' 'florina' 'tortoli' 'limnos' 'gaeta' 'borgarello'
 'pizzighettone e formigara' 'cork' 'limerick' 'bangkok' 'galway'
 'bielsko-biala' 'montecatini terme' 'bydgoszcz' 'london' 'aspen, co'
 'carrara' 'reykjavik' 'hamburg' 'sestri levante - lavagna' 'arezzo'
 'astana' 'paris' 'ascoli piceno' 'bergamo' 'girona' 'alba' 'milano'
 'biella' 'chivasso' 'zaragoza' 'cd de mexico' 'busto arsizio' 'antwerpen'
 'cuneo' 'malmo' 'las condes' 'madrid' 'didymoteicho' 'frosinone'
 'glyfada' 'igoumenitsa' 'lecce' 'ioannina' 'karditsa' 'manerba del garda'
 'kavala' 'kinouria' 'komotini' 'patra' 'marciana marina' 'rethymno'
 'samos' 'novara' 'nicosia' 'mani' 'irakleio' 'thessaloniki' 'athens'
 'padova' 'sao paulo' 'a coruna' 'ferrol - naron'
 'curtin university, perth, wa' 'melbourne, au' 'ferrara' 'montreal, qc'


In [16]:
# get list of endpoints for the specified city
hrefs = df_networks.loc[[True if 'boston' in city else False for city in city_list ],'href']
N_hrefs = hrefs.shape[0]
print(N_hrefs)

1


In [20]:
# set up a dataframe to store stations data
df = pd.DataFrame()

In [17]:
# go through each endpoint (there will likely only be one)
for href in hrefs.values:
    print("http://api.citybik.es" + href)

http://api.citybik.es/v2/networks/blue-bikes


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [18]:
# get the stations data for that endpoint
response = requests.request('GET',url="http://api.citybik.es" + href + "?fields=stations")
data = response.json()
df1 = pd.json_normalize(data)

print(df1)

                                    network.stations
0  [{'empty_slots': 4, 'extra': {'ebikes': 0, 'ha...


Put your parsed results into a DataFrame.

In [21]:
# extract the stations only and add to the dataframe
stations = data['network']['stations']
df_new = pd.json_normalize(stations)
df = pd.concat([df, df_new])
print(df)

     empty_slots  free_bikes                                id   latitude  \
0              4          15  553ed0300d38108b4f21a6bafa3db70c  42.386781   
1             24           8  93542dcbf21f5411569adb92cd7cc199  42.368844   
2              6           8  d9c7ef5dbda4ed944d1bf51fe540acb6  42.369536   
3             13           1  0568389e659e679fbe29a5ac12cd49c0  42.383533   
4              7           8  47b79abc28a54d0e4689b1096ceb8466  42.373312   
..           ...         ...                               ...        ...   
450            5           6  ed6f34cef0905446b3cf010bd30fb270  42.528101   
451            8           7  a6390deb3a13421507905ebf409f68e1  42.420349   
452           14           1  74b552449ea993462ca29ce629988081  42.375354   
453           11           3  3e478957db03b9bca59a95702ee09a9a  42.364475   
454            7           4  7faf5d041784946b648cd3c617e8a716  42.374106   

     longitude                                               name  \
0   -7

In [22]:
df.head(10)

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.payment,extra.payment-terminal,extra.renting,extra.returning,extra.slots,extra.uid
0,4,15,553ed0300d38108b4f21a6bafa3db70c,42.386781,-71.006098,Orient Heights T Stop - Bennington St at Sarat...,2023-07-01T15:09:55.067000Z,0,True,1688218107,"[creditcard, key]",True,1,1,19,217
1,24,8,93542dcbf21f5411569adb92cd7cc199,42.368844,-71.039778,Maverick Square - Lewis Mall,2023-07-01T15:09:55.075000Z,0,True,1688221619,"[creditcard, key]",True,1,1,33,212
2,6,8,d9c7ef5dbda4ed944d1bf51fe540acb6,42.369536,-71.039431,East Boston Neighborhood Health Center - 20 Ma...,2023-07-01T15:09:55.076000Z,0,True,1688221565,"[creditcard, key]",True,1,1,16,213
3,13,1,0568389e659e679fbe29a5ac12cd49c0,42.383533,-71.016191,Bennington St at Byron St,2023-07-01T15:09:55.077000Z,0,True,1688199882,"[creditcard, key]",True,1,1,15,210
4,7,8,47b79abc28a54d0e4689b1096ceb8466,42.373312,-71.041020,Boston East - 126 Border St,2023-07-01T15:09:55.079000Z,0,True,1688208427,"[creditcard, key]",True,1,1,15,219
5,13,10,68fc84a359655b54175ddc443e79003b,42.310579,-71.107341,Green Street T Stop - Green St at Amory St,2023-07-01T15:09:54.835000Z,0,True,1688221126,"[creditcard, key]",True,1,1,23,133
6,16,3,3831590999fc4a7f52d08952a6bd045d,42.361780,-71.108100,Dana Park,2023-07-01T15:09:54.841000Z,0,True,1688223984,"[creditcard, key]",True,1,1,19,139
7,10,2,af050e8bab271eeebf49b5407a09d415,42.326599,-71.066498,Newmarket Square T Stop - Massachusetts Ave at...,2023-07-01T15:09:54.842000Z,0,True,1688223115,"[creditcard, key]",True,1,1,15,138
8,6,13,1b7f50a75403e1f818b0b1dd81c09e9f,42.371197,-71.097599,Cambridge Dept. of Public Works -147 Hampshire...,2023-07-01T15:09:55.122000Z,0,True,1688224094,"[creditcard, key]",True,1,1,19,225
9,6,8,a45aabe9b54b087818818dc78372bc43,42.341332,-71.076847,South End Library - Tremont St at W Newton St,2023-07-01T15:09:54.847000Z,0,True,1688221463,"[creditcard, key]",True,1,1,15,25


In [23]:
df.tail(10)

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.payment,extra.payment-terminal,extra.renting,extra.returning,extra.slots,extra.uid
445,12,2,a2cdc74d5b6c98782800e4c32a4a219a,42.363565,-71.100442,Mass Ave/Lafayette Square,2023-07-01T15:09:54.756000Z,0,True,1688223756,"[creditcard, key]",True,1,1,15,597
446,20,5,8586f37d0cc61be306d6c1b5338ccc89,42.336610,-71.110047,Riverway at Brookline Ave,2023-07-01T15:09:54.755000Z,0,True,1688223467,"[creditcard, key]",True,1,1,25,594
447,4,7,2db8b3b9c885715c8132c0cf2e184721,42.486288,-70.901667,Vinnin Square,2023-07-01T15:09:54.963000Z,0,True,1688202070,"[creditcard, key]",True,1,1,11,598
448,1,10,559c06f46de4069ae78d87bb2ca6649b,42.520096,-70.891390,Derby Street,2023-07-01T15:09:54.771000Z,0,True,1688223432,"[creditcard, key]",True,1,1,11,599
449,1,16,24e99d91d84351d3e224e3d7ef2c7799,42.322931,-71.100141,Jackson Square T Stop,2023-07-01T15:09:54.836000Z,0,True,1688221860,"[creditcard, key]",True,1,1,18,131
450,5,6,ed6f34cef0905446b3cf010bd30fb270,42.528101,-70.889711,Bridge Street Neck,2023-07-01T15:09:54.892000Z,0,True,1688214785,"[creditcard, key]",True,1,1,11,588
451,8,7,a6390deb3a13421507905ebf409f68e1,42.420349,-71.044198,Chelsea St at Vine St,2023-07-01T15:09:54.878000Z,0,True,1688202028,"[creditcard, key]",True,1,1,15,395
452,14,1,74b552449ea993462ca29ce629988081,42.375354,-71.031318,Airport T Stop - Bremen St at Brooks St,2023-07-01T15:09:55.068000Z,0,True,1688202052,"[creditcard, key]",True,1,1,15,214
453,11,3,3e478957db03b9bca59a95702ee09a9a,42.364475,-71.128408,175 N Harvard St,2023-07-01T15:09:54.834000Z,0,True,1688223924,"[creditcard, key]",True,1,1,17,149
454,7,4,7faf5d041784946b648cd3c617e8a716,42.374106,-71.032763,Bremen St at Marion St,2023-07-01T15:09:55.128000Z,0,True,1688220469,"[creditcard, key]",True,1,1,11,600


In [24]:
df.count()

empty_slots               455
free_bikes                455
id                        455
latitude                  455
longitude                 455
name                      455
timestamp                 455
extra.ebikes              455
extra.has_ebikes          455
extra.last_updated        455
extra.payment             455
extra.payment-terminal    455
extra.renting             455
extra.returning           455
extra.slots               455
extra.uid                 455
dtype: int64